In [41]:
!pip install soynlp

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt, Mecab
from soynlp.word import WordExtractor

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [3]:
data = pd.read_csv('/aiffel/train_1000_mk2.csv')
data.head()

,non_label_sentence,label_sentence,class,binary_class
0,역시 일기예보는 맞은 적이 한 번도 없어\n요즘 잘 맞는 거 같던데\n어제 오늘 비...,1:역시 일기예보는 맞은 적이 한 번도 없어\n2:요즘 잘 맞는 거 같던데\n1:어...,일반 대화,일반 대화
1,요즘엔 군대도 많이 편해 졌다고 하더라\n맞아\n다들 그 얘기 많이 하더라\n모든 ...,1:요즘엔 군대도 많이 편해 졌다고 하더라\n2:맞아\n2:다들 그 얘기 많이 하더...,일반 대화,일반 대화
2,나 피아노 학원이나 다닐까\n갑자기 뭔 놈에 피아노 학원\n안 친 세월이 몇 년이야...,1:나 피아노 학원이나 다닐까\n2:갑자기 뭔 놈에 피아노 학원\n1:안 친 세월이...,일반 대화,일반 대화
3,우리 강아지가 자꾸 아무데나 똥을 싸\n교육을 시켜야겠다\n배변 훈련은 어디서 시켜...,1:우리 강아지가 자꾸 아무데나 똥을 싸\n2:교육을 시켜야겠다\n1:배변 훈련은 ...,일반 대화,일반 대화
4,언니 혼내는 거 아녀요\n원래 혼내면서 배우는 거지\n난 솔직히 아소비 보내고 싶...,1:언니 혼내는 거 아녀요\n2:원래 혼내면서 배우는 거지\n1:난 솔직히 아소비 ...,일반 대화,일반 대화


In [4]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer
vocab_size = 30000

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

sentences = [tokenizer.tokenize(sen) for sen in sentences]
tokenizer_tf = Tokenizer(num_words=vocab_size)
tokenizer_tf.fit_on_texts(sentences)
word_dic = tokenizer_tf.word_index
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)
np.shape(padded)

training was done. used memory 0.489 Gbry 0.412 Gb
all cohesion probabilities was computed. # words = 868
all branching entropies was computed # words = 23278
all accessor variety was computed # words = 23278


(4870, 191)

In [5]:
len(word_dic)

48800

In [6]:
train_data = padded
train_label = data['class']
print(len(train_data), len(train_label))

4870 4870


In [41]:
labels = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}

In [7]:
labels = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3896 487 487
3896 487 487


# 모델 
- LSTM 사용

In [8]:
word_vector_dim = 1024
labels_size = len(labels)
hidden_size = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size//2))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 1024)        30720000  
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         590336    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 30000)             1950000   
_________________________________________________________________
dropout (Dropout)            (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
16/16 [==============================] - 8s 169ms/step - loss: 1.3803 - accuracy: 0.3568 - val_loss: 0.9637 - val_accuracy: 0.5708
Epoch 2/10
16/16 [==============================] - 2s 109ms/step - loss: 0.7025 - accuracy: 0.7017 - val_loss: 0.7316 - val_accuracy: 0.7269
Epoch 3/10
16/16 [==============================] - 2s 112ms/step - loss: 0.2734 - accuracy: 0.9032 - val_loss: 0.7401 - val_accuracy: 0.7474
Epoch 4/10
16/16 [==============================] - 2s 110ms/step - loss: 0.1034 - accuracy: 0.9666 - val_loss: 1.0191 - val_accuracy: 0.7515
Epoch 5/10
16/16 [==============================] - 2s 112ms/step - loss: 0.0367 - accuracy: 0.9877 - val_loss: 1.1399 - val_accuracy: 0.7536
Epoch 6/10
16/16 [==============================] - 2s 112ms/step - loss: 0.0179 - accuracy: 0.9961 - val_loss: 1.0513 - val_accuracy: 0.7803
Epoch 7/10
16/16 [==============================] - 2s 111ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 1.5171 - val_accuracy: 0.7598
Epoch 

In [10]:
test_file_path = '/aiffel/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
    
test_data = test_dataset.transpose()
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [19]:
sentences = [sen for sen in test_data['text']]
sentences = [tokenizer.tokenize(sen) for sen in sentences]
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)

In [21]:
np.shape(padded)

(500, 172)

In [35]:
pred = model.predict(padded)

from sklearn.metrics import classification_report

pred_label = []

for i in range(len(pred)):
    pred_label.append((str)(np.argmax(pred[i])))
    
pred_label[:10]

['0', '2', '2', '4', '1', '0', '0', '1', '4', '1']

In [36]:
label_change = {'0':'00', '1':'01', '2':'02', '3':'03', '4':'04'}

sub_label = []

for pre in pred_label:
    sub_label.append(label_change[pre])

sub_label[:10]

['00', '02', '02', '04', '01', '00', '00', '01', '04', '01']

In [37]:
submission = test_data.assign(CLASS=sub_label)
submission = submission.rename(columns={'CLASS':'class'})

In [38]:
submission.head()

,text,class
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,00
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,02
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,02
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,04
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,01


In [52]:
test_data['text'][6]

'그 사람 누구냐?  누구말하는거야?  다 알면서 모른척 하지마. 둘다 쏴버리기 전에.  아니 정말 왜이래? 누굴말하는거냐고.  어제 밤에 전화온 그새끼 말이야. 번호 알아놨으니까 죽이기 전에 다 불어라.  아! 김대리? 그냥 회사사람이야. 오해하지마  회사사람이란 인간이 그 오밤중에 사랑한다고 보고싶다고 하냐? 진짜 다 죽여버리기 전에 솔직히 말해라.  정말 아니야. 왜 이래? 오해라니까.  안되겠다. 그럼 지난주 점심시간에 모텔은 왜갔냐?  어??? 그냥 피곤해서 쉬러간거야. 오해야.  안되겠다. 그냥 불지르고 다 끝내자.'

In [42]:
labels

{'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}

In [39]:
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,00,02,02,04,01,00,00,01,04,01,...,00,04,00,01,01,02,02,01,02,00


In [53]:
import json

submission_file_path = '/aiffel/submission_LSTM_1000_수정'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)